### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [1]:
import pandas as pd
import requests as re
from shapely.geometry import Point

In [2]:
#LEEMOS NUESTRO FICHERO
bicimad_df = pd.read_csv('../data/bicimad_stations.csv', sep = '\t')

In [3]:
#bicimad_df.info()

In [4]:
#bicimad_df.info()

In [5]:
'''

bicimad_df['longitude'] = bicimad_df.apply(lambda row: row[
    'geometry.coordinates'].strip('[]').split(', ')[0], 
                                           axis=1)
bicimad_df['latitude'] = bicimad_df.apply(lambda row: row[
    'geometry.coordinates'].strip('[]').split(', ')[1], 
                                          axis=1)
'''

"\n\nbicimad_df['longitude'] = bicimad_df.apply(lambda row: row[\n    'geometry.coordinates'].strip('[]').split(', ')[0], \n                                           axis=1)\nbicimad_df['latitude'] = bicimad_df.apply(lambda row: row[\n    'geometry.coordinates'].strip('[]').split(', ')[1], \n                                          axis=1)\n"

In [37]:

#JUNTAMOS TODO EN UNA LINEA DE COMANDO PARA QUE PASE SOLO 1 VEZ POR EL DataFrame con pd.Series
bicimad_df[['longitude', 'latitude']] = bicimad_df.apply(lambda row: pd.Series(
    [row['geometry.coordinates'].strip('[]').split(', ')[0], 
     row['geometry.coordinates'].strip('[]').split(', ')[1]]), 
                                                           axis=1)

CPU times: user 41.2 ms, sys: 2.16 ms, total: 43.4 ms
Wall time: 42.4 ms


In [7]:
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
 14  longitude             264 non-null    object
 15  latitude              264 non-null    ob

In [8]:
bicipark_df = pd.read_csv('../data/bicipark_stations.csv', sep=';')

In [35]:
%%time
#SEPARADO
bicipark_df['longitude'] = bicipark_df.apply(lambda row: row[
'geometry.coordinates'].strip('[]').split(', ')[0], axis=1)

bicipark_df['latitude'] = bicipark_df.apply(lambda row: row[
'geometry.coordinates'].strip('[]').split(', ')[1], axis=1)

CPU times: user 1.37 ms, sys: 51 µs, total: 1.43 ms
Wall time: 1.39 ms


In [36]:
%%time
#JUNTAMOS TODO EN UNA LINEA DE COMANDO PARA QUE PASE SOLO 1 VEZ POR EL DataFrame con pd.Series
bicipark_df[['longitude', 'latitude']] = bicipark_df.apply(lambda row: pd.Series(
    [row['geometry.coordinates'].strip('[]').split(', ')[0], 
     row['geometry.coordinates'].strip('[]').split(', ')[1]]), 
                                                           axis=1)

CPU times: user 2.68 ms, sys: 26 µs, total: 2.71 ms
Wall time: 2.69 ms


In [11]:
bicipark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            11 non-null     int64 
 1   stationId             11 non-null     int64 
 2   stationName           11 non-null     object
 3   address               11 non-null     object
 4   city                  11 non-null     object
 5   zip_code              11 non-null     object
 6   total_places          11 non-null     int64 
 7   free_places           11 non-null     int64 
 8   enabled               11 non-null     int64 
 9   reserved_places       11 non-null     int64 
 10  geometry.type         11 non-null     object
 11  geometry.coordinates  11 non-null     object
 12  longitude             11 non-null     object
 13  latitude              11 non-null     object
dtypes: int64(6), object(8)
memory usage: 1.3+ KB


In [12]:
bicipark_clean = bicipark_df[['stationId', 'stationName', 'address','longitude', 'latitude']]
bicipark_clean.tail(3)
print(bicipark_clean.columns)

Index(['stationId', 'stationName', 'address', 'longitude', 'latitude'], dtype='object')


In [13]:
#CONECTAMOS CON LA WEB PARA OBTENER JSON
base_url = "https://datos.madrid.es/egob"
body = "/catalogo/209434-0-templos-otros.json"
response = re.get(base_url + body)
print(response)

<Response [200]>


In [14]:
#ANALIZAMOS JSON TEMPLOS
content = response.content
json_data = response.json()
json_data.keys()

dict_keys(['@context', '@graph'])

In [15]:
json_data.keys()

dict_keys(['@context', '@graph'])

In [16]:
templos = pd.DataFrame(json_data['@graph'])

In [17]:
def separator_longitud_json(column): #devuelve valor como float64
    for i in column:
        return column["longitude"]
def separator_latitud_json(column): #devuelve valor como float64
    for i in column:
        return column["latitude"]

In [18]:
templos['longitude'] = templos.apply(lambda row: separator_longitud_json(row["location"]), axis=1)
templos['latitude'] = templos.apply(lambda row: separator_latitud_json(row["location"]), axis=1)

In [19]:
templos.head()

,@id,@type,id,title,relation,address,location,organization,longitude,latitude
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11772877,Asamblea Cristiana Gilgal,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.38724931072868, 'longitude': -...",{'organization-desc': 'Templo evangélico. Bus...,-3.638521,40.387249
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11025958,Asambleas de Evangelización Mundial para Crist...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.372800292475574, 'longitude': ...",{'organization-desc': 'Templo evangélico Metr...,-3.745074,40.372800
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11026848,Asociación Evangélica Internacional,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.457895405861926, 'longitude': ...",{'organization-desc': 'Templo evangélico. Met...,-3.670675,40.457895
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3387,Catedral Anglicana de Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42635538209311, 'longitude': -...",{'organization-desc': 'Iglesia Española Reform...,-3.698666,40.426355
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,3388,Catedral Ortodoxa Griega. Santa Iglesia Catedr...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.45755529500435, 'longitude': -...",{'organization-desc': ' Metro: Colombia (líne...,-3.672895,40.457555


In [20]:
templos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   @id           298 non-null    object 
 1   @type         298 non-null    object 
 2   id            298 non-null    object 
 3   title         298 non-null    object 
 4   relation      298 non-null    object 
 5   address       298 non-null    object 
 6   location      298 non-null    object 
 7   organization  298 non-null    object 
 8   longitude     298 non-null    float64
 9   latitude      298 non-null    float64
dtypes: float64(2), object(8)
memory usage: 23.4+ KB


In [21]:
templos_clean = templos[['id','title','longitude','latitude']]
templos_clean.tail(3)

,id,title,longitude,latitude
295,11778015,Unión de Iglesias Cristianas Adventistas del 7...,-3.751562,40.389475
296,11744860,Unión de Iglesias Cristianas Adventistas del 7...,-3.738156,40.394823
297,11773010,Yoido Full Gospel Madrid Church,-3.709605,40.460551


In [22]:
bicimad_clean = bicimad_df[['id','name','address','longitude','latitude']]

In [23]:
bicimad_clean.head(2)

,id,name,address,longitude,latitude
0,1,1a - Puerta del Sol A,Puerta del Sol nº 1,-3.7018341,40.4172137
1,2,1b - Puerta del Sol B,Puerta del Sol nº 1,-3.701602938060457,40.41731271011562


In [24]:
"""
bicimad_clean 'id', 'name', 'address', 'longitude', 'latitude'

bicipark_clean 'stationId', 'stationName', 'longitude', 'latitude'

templos_clean 'id', 'title', 'address', 'longitude', 'latitude'

"""

"\nbicimad_clean 'id', 'name', 'address', 'longitude', 'latitude'\n\nbicipark_clean 'stationId', 'stationName', 'longitude', 'latitude'\n\ntemplos_clean 'id', 'title', 'address', 'longitude', 'latitude'\n\n"

In [25]:
from geopy.distance import geodesic

In [26]:
# CREAMOS UN NUEVO DATAFRAME
templos_con_distancia = templos_clean.copy()

templos_con_distancia.head()

,id,title,longitude,latitude
0,11772877,Asamblea Cristiana Gilgal,-3.638521,40.387249
1,11025958,Asambleas de Evangelización Mundial para Crist...,-3.745074,40.372800
2,11026848,Asociación Evangélica Internacional,-3.670675,40.457895
3,3387,Catedral Anglicana de Madrid,-3.698666,40.426355
4,3388,Catedral Ortodoxa Griega. Santa Iglesia Catedr...,-3.672895,40.457555


In [27]:
# CREAMOS UN NUEVO DATAFRAME
templos_con_distancia = templos_clean.copy()

In [28]:
%%time
# WINNER!!!!!
templos_con_distancia['distancia_minima'] = templos_con_distancia.apply(lambda row: min(
    geodesic((row['latitude'], row['longitude']), (line['latitude'], line['longitude'])).meters
    for _, line in bicimad_clean.iterrows()), axis=1).copy()



CPU times: user 17.2 s, sys: 88.2 ms, total: 17.3 s
Wall time: 17.3 s


In [29]:
templos_con_distancia.head(300)

,id,title,longitude,latitude,distancia_minima
0,11772877,Asamblea Cristiana Gilgal,-3.638521,40.387249,1494.529207
1,11025958,Asambleas de Evangelización Mundial para Crist...,-3.745074,40.372800,3719.349550
2,11026848,Asociación Evangélica Internacional,-3.670675,40.457895,486.250462
3,3387,Catedral Anglicana de Madrid,-3.698666,40.426355,149.990785
4,3388,Catedral Ortodoxa Griega. Santa Iglesia Catedr...,-3.672895,40.457555,294.549185
...,...,...,...,...,...
293,11768905,Unión de Iglesias Cristianas Adventistas del 7...,-3.608022,40.424746,2851.695686
294,7065947,Unión de Iglesias Cristianas Adventistas del 7...,-3.649838,40.430251,295.869712
295,11778015,Unión de Iglesias Cristianas Adventistas del 7...,-3.751562,40.389475,2930.645985
296,11744860,Unión de Iglesias Cristianas Adventistas del 7...,-3.738156,40.394823,1768.087468


In [30]:
templos_con_distancia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                298 non-null    object 
 1   title             298 non-null    object 
 2   longitude         298 non-null    float64
 3   latitude          298 non-null    float64
 4   distancia_minima  298 non-null    float64
dtypes: float64(3), object(2)
memory usage: 11.8+ KB


In [31]:
templos_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         298 non-null    object 
 1   title      298 non-null    object 
 2   longitude  298 non-null    float64
 3   latitude   298 non-null    float64
dtypes: float64(2), object(2)
memory usage: 9.4+ KB
